In [ ]:
import logging
from copy import deepcopy

import astropy.units as u
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
from torch import nn
from torch.utils.data import DataLoader, TensorDataset
from tqdm import tqdm
from unyt import Jy, nJy

# def smooth_sign(x, k=100.):
#     return 2. * torch.sigmoid(k * x) - 1


def parity_odd_power(x, alpha=2):
    """Applies the parity odd power transformation to the input tensor."""
    return x * (torch.abs(x) ** (alpha - 1))


def leaky_parity_odd_power(x, alpha=2):
    """Applies the leaky parity odd power transformation to the input tensor."""
    return x + parity_odd_power(x, alpha)


class POPExpLoss(nn.Module):
    """Exponential loss function for evidence network training."""

    def forward(self, model_pred, model_label):
        """Exponential loss function for evidence network training."""
        model_pred = leaky_parity_odd_power(model_pred, alpha=1)
        model_pred = torch.clamp(model_pred, -50, 50)
        loss_val = torch.exp((0.5 - model_label) * model_pred)
        return torch.mean(loss_val)


class ExpLoss(nn.Module):
    """Exponential loss function for evidence network training."""

    def forward(self, model_pred, model_label):
        """Exponential loss function for evidence network training."""
        loss_val = (0.5 - model_label) * model_pred
        out = torch.logsumexp(loss_val, dim=0)
        return out


class EvidenceNetwork(nn.Module):
    """A deep neural network designed to approximate the log-Bayes factor.

    The architecture consists of an optional embedding network, followed by an
    initial fully-connected layer, a series of residual blocks, and a final
    output layer. This structure allows the network to learn complex features from
    the input data.

    Args:
        input_size (int): The dimension of the input features that are fed into the
            first fully-connected layer. This should match the output dimension
            of the `embedding_net`.
        embedding_net (nn.Module, optional): A PyTorch module to preprocess or
            embed the raw input data before it enters the main network. Defaults
            to nn.Identity().
        layer_width (int, optional): The number of neurons in the hidden layers.
            Defaults to 16.
        added_layers (int, optional): The number of residual blocks to include in
            the network. Defaults to 3.
        batch_norm_flag (int, optional): A flag to enable (1) or disable (0)
            batch normalization. Defaults to 1.
        alpha (int, optional): The exponent for the `leaky_parity_odd_power`
            activation function applied to the output. Defaults to 2.
    """

    def __init__(
        self,
        input_size,
        embedding_net=nn.Identity(),
        layer_width=16,
        added_layers=3,
        batch_norm_flag=1,
        alpha=2,
    ):
        """Initializes the EvidenceNetwork with the specified architecture."""
        super().__init__()
        self.input_size = input_size
        self.embedding_net = embedding_net
        self.layer_width = layer_width
        self.added_layers = added_layers
        self.bn = batch_norm_flag
        self.alpha = alpha

        self.initial_layer = self.simple_layer(self.input_size, self.layer_width, 1)

        self.hidden_layers = nn.ModuleList(
            [self.simple_layer(self.layer_width, self.layer_width, 1) for _ in range(1)]
        )

        self.residual_layers = nn.ModuleList(
            [
                self.residual_layer(self.layer_width, self.layer_width, self.bn)
                for _ in range(self.added_layers)
            ]
        )

        self.post_residual = self.simple_layer(
            self.layer_width, self.layer_width, batch_norm_flag=0
        )
        self.output_layer = nn.Linear(self.layer_width, 1)

    def simple_layer(self, in_features, out_features, batch_norm_flag=1):
        """Creates a simple fully-connected layer with LeakyReLU activation."""
        layers = [nn.Linear(in_features, out_features), nn.LeakyReLU(0.1)]
        if batch_norm_flag == 1:
            layers.append(nn.BatchNorm1d(self.layer_width))
        return nn.Sequential(*layers)

    def residual_layer(self, in_features, out_features, batch_norm_flag=1):
        """Creates a residual block with two fully-connected layers."""
        layers = [
            self.simple_layer(in_features, out_features, batch_norm_flag),
            self.simple_layer(out_features, out_features, batch_norm_flag),
        ]
        return nn.Sequential(*layers)

    def forward(self, x):
        """Forward pass through the EvidenceNetwork."""
        # Embedding Network
        x = self.embedding_net(x)

        # Initial layer (resizing input to layer width)
        x = self.initial_layer(x)

        # Hidden layers
        for layer in self.hidden_layers:
            x = layer(x)

        # Residual layers
        for layer in self.residual_layers:
            x = x + layer(x)

        # Output layer
        x = self.post_residual(x)
        x = self.output_layer(x)
        x = 0.1 * x + 0.001
        x = leaky_parity_odd_power(x, alpha=self.alpha)
        return x


class K_EvidenceNetwork:
    """A runner class to train an EvidenceNetwork for approximating the Bayes Factor.

    This class handles data preparation, training loops, validation, early stopping,
    and model saving. It can optionally use a custom embedding network to process
    complex data modalities before the main fully-connected layers.

    Args:
        embedding_net (nn.Module, optional): A PyTorch module to preprocess input
            data. If None, an identity mapping is used. Defaults to None.
        layer_width (int, optional): The number of neurons in the hidden layers of
            the EvidenceNetwork. Defaults to 16.
        added_layers (int, optional): The number of residual blocks in the
            EvidenceNetwork. Defaults to 3.
        batch_norm_flag (int, optional): A flag to enable (1) or disable (0)
            batch normalization. Defaults to 1.
        alpha (int, optional): The exponent for the leaky_parity_odd_power
            activation function. Defaults to 2.
        train_args (dict, optional): A dictionary of training hyperparameters to
            override the defaults. Defaults to {}.
        device (str, optional): The device to run training on (e.g., 'cpu' or
            'cuda'). Defaults to 'cpu'.

    Attributes:
        model (EvidenceNetwork): The trained PyTorch model. This is None until
            the `train` method is successfully called.
        best_val (float): The best validation loss achieved during training.
        loss_fn (nn.Module): The loss function instance used for training.
    """

    def __init__(
        self,
        embedding_net=None,
        layer_width=16,
        added_layers=3,
        batch_norm_flag=1,
        alpha=2,
        train_args={},
        device="cpu",
    ):
        """Initializes the K_EvidenceNetwork with specified architecture and training parameters."""
        self.embedding_net = embedding_net or nn.Identity()
        self.layer_width = layer_width
        self.added_layers = added_layers
        self.batch_norm_flag = batch_norm_flag
        self.alpha = alpha
        self.train_args = dict(
            training_batch_size=32,
            learning_rate=1e-5,
            stop_after_epochs=30,
            clip_max_norm=5,
            max_epochs=int(1e4),
            validation_fraction=0.1,
        )
        self.train_args.update(train_args)
        self.device = device

        self.embedding_net.to(self.device)

        self.loss_fn = ExpLoss()
        self.best_val = float("inf")
        self.model = None

    def _loss(self, model, theta, x):
        """Compute the loss function for a given model."""
        logK = model(x)
        return self.loss_fn(logK, theta.view(-1, 1))

    def _train_epoch(self, model, train_loader, val_loader, optimizer):
        """Train a single epoch of a neural network model."""
        model.train()
        loss_train, count_train = [], 0
        for x, theta in train_loader:
            x, theta = x.to(self.device), theta.to(self.device)
            optimizer.zero_grad()
            loss = self._loss(model, theta, x)
            loss.backward()

            norm = nn.utils.clip_grad_norm_(model.parameters(), self.train_args["clip_max_norm"])

            if torch.isfinite(norm):
                optimizer.step()
            # Record
            loss_train.append(loss.item() * len(theta))
            count_train += len(theta)
        loss_train = sum(loss_train) / count_train

        model.eval()
        with torch.no_grad():
            loss_val, count_val = [], 0
            for x, theta in val_loader:
                x, theta = x.to(self.device), theta.to(self.device)
                loss_val.append(self._loss(model, theta, x).item() * len(theta))
                count_val += len(theta)
        loss_val = sum(loss_val) / count_val
        return loss_train, loss_val

    def train(self, loader1, loader2, show_progress_bars=True):
        """Trains the evidence network.

        Args:
            loader1: A data loader object for the first model. Must have a
                     `get_all_data()` method.
            loader2: A data loader object for the second model.
            show_progress_bars (bool): Whether to display tqdm progress bars.
        """
        # Aggregate data from different models, and label them
        # Assumes loaders have a 'get_all_data' method returning numpy arrays
        x1 = loader1.get_all_data().astype(np.float32)
        x2 = loader2.get_all_data().astype(np.float32)
        x = np.concatenate([x1, x2], axis=0)
        labels = np.concatenate([np.zeros(len(x1)), np.ones(len(x2))], axis=0)

        # Train/Validation split
        indices = np.random.permutation(len(x))
        val_size = int(len(x) * self.train_args["validation_fraction"])
        val_indices, train_indices = indices[:val_size], indices[val_size:]
        x_train, labels_train = x[train_indices], labels[train_indices]
        x_val, labels_val = x[val_indices], labels[val_indices]

        # Create data loaders
        train_data = TensorDataset(
            torch.tensor(x_train, dtype=torch.float32),
            torch.tensor(labels_train, dtype=torch.float32),
        )
        val_data = TensorDataset(
            torch.tensor(x_val, dtype=torch.float32), torch.tensor(labels_val, dtype=torch.float32)
        )
        train_loader = DataLoader(
            train_data,
            batch_size=self.train_args["training_batch_size"],
            shuffle=True,
            drop_last=True,
        )
        val_loader = DataLoader(
            val_data,
            batch_size=self.train_args["training_batch_size"],
            shuffle=False,
            drop_last=True,
        )

        # Determine the input dimension for the dense network
        if not isinstance(self.embedding_net, nn.Identity):
            with torch.no_grad():
                # Use a single-item batch to find the output dimension
                sample_input = next(iter(train_loader))[0].to(self.device)
                embedded_output = self.embedding_net(sample_input)
                ndim = embedded_output.shape[-1]
        else:
            # If no embedding net, the input dim is from the data itself
            ndim = x_train.shape[-1]

        # Create model, passing the embedding net
        model = EvidenceNetwork(
            input_size=ndim,
            embedding_net=self.embedding_net,
            layer_width=self.layer_width,
            added_layers=self.added_layers,
            batch_norm_flag=self.batch_norm_flag,
            alpha=self.alpha,
        )
        model.to(self.device)
        optimizer = torch.optim.Adam(model.parameters(), lr=self.train_args["learning_rate"])

        # Train model
        wait = 0
        summary = {"training_loss": [], "validation_loss": []}
        best_model_state = deepcopy(model.state_dict())

        with tqdm(
            iter(range(self.train_args["max_epochs"])),
            unit=" epochs",
            disable=not show_progress_bars,
        ) as tq:
            for epoch in tq:
                loss_train, loss_val = self._train_epoch(
                    model=model,
                    train_loader=train_loader,
                    val_loader=val_loader,
                    optimizer=optimizer,
                )
                if show_progress_bars:
                    tq.set_postfix(loss_train=f"{loss_train:.4f}", loss_val=f"{loss_val:.4f}")

                summary["training_loss"].append(loss_train)
                summary["validation_loss"].append(loss_val)

                if loss_val < self.best_val:
                    self.best_val = loss_val
                    best_model_state = deepcopy(model.state_dict())
                    wait = 0
                elif wait >= self.train_args["stop_after_epochs"]:
                    logging.info(f"Stopping early after {epoch} epochs.")
                    break
                else:
                    wait += 1
            else:
                logging.warning(
                    f"Training did not converge in {self.train_args['max_epochs']} epochs."
                )

        summary["best_validation_loss"] = self.best_val
        summary["epochs_trained"] = epoch + 1
        self.model = model
        self.model.load_state_dict(best_model_state)
        return summary

    def predict(self, x):
        """Predict the log-Bayes Ratio for a given input."""
        if self.model is None:
            raise ValueError("Model has not been trained yet. Please call .train() first.")
        self.model.eval()
        with torch.no_grad():
            x_tensor = torch.from_numpy(np.atleast_2d(x).astype(np.float32)).to(self.device)
            return self.model(x_tensor).cpu().numpy()


def plot_histogram(logK_bpass, logK_pop3):
    """Plot histogram of log Bayes factors for two models with Jeffreys scale annotations."""
    plt.figure(figsize=(10, 6))

    jeffries_logK = {
        (-100, -2): "deciseivly against",
        (-2, -1.5): "very strongly against",
        (-1.5, -1): "strongly against",
        (-1, -0.5): "moderately against",
        (-0.5, 0.5): "No strong evidence",
        (0.5, 1): "moderately for",
        (1, 1.5): "strongly for",
        (1.5, 2): "very strongly for",
        (2, 100): "decisively for",
    }

    # Log-spaced bins from -100 to 100
    bins = np.concatenate((-np.logspace(0, 2, 20)[::-1], np.logspace(0, 2, 20)))
    plt.hist(logK_bpass, bins=bins, alpha=0.5, label="BPASS", color="C0")
    plt.hist(logK_pop3, bins=bins, alpha=0.5, label="Pop III", color="C1")
    plt.axvline(0, color="k", linestyle="--", alpha=0.4)
    plt.xlabel("log K (Pop III / BPASS) ")
    plt.ylabel("Number of samples")

    # add shaded regions with text (white text on dark background)
    for pos, ((x0, x1), label) in enumerate(jeffries_logK.items()):
        plt.axvspan(x0, x1, color="gray", alpha=0.2 if pos % 2 == 0 else 0.1)
        plt.text(
            (x0 + x1) / 2,
            plt.ylim()[1] * 0.9,
            label,
            color="black" if pos % 2 == 0 else "dimgray",
            ha="center",
            va="top",
            fontsize=8,
            rotation=90,
        )

    # Add a symlog scale on the x-axis
    plt.xscale("symlog", linthresh=3)

    # Label either end of the x-axis with the corresponding model
    plt.text(-90, plt.ylim()[1] * 0.94, "BPASS", color="C0", fontsize=12, ha="left")
    plt.text(90, plt.ylim()[1] * 0.94, "PopIII", color="C1", fontsize=12, ha="right")

    plt.legend()

In [ ]:
def plot_histogram(logK_X, logK_Y, Y_label="Model Y", X_label="Model X"):
    """Plot histogram of log Bayes factors for two models with Jeffreys scale annotations."""
    plt.figure(figsize=(10, 6))

    jeffries_logK = {
        (-100, -2): "decisively against",
        (-2, -1.5): "very strongly against",
        (-1.5, -1): "strongly against",
        (-1, -0.5): "moderately against",
        (-0.5, 0.5): "No strong evidence",
        (0.5, 1): "moderately for",
        (1, 1.5): "strongly for",
        (1.5, 2): "very strongly for",
        (2, 100): "decisively for",
    }

    # Jeffries bins (with more detail between -100 and 2 and 2 and 100)
    bins = np.concatenate(
        [
            -np.logspace(0.31, 2, 20)[::-1],
            np.linspace(-2, 2, 20),
            np.logspace(0.31, 2, 20),
        ]
    )

    plt.hist(logK_X, bins=bins, alpha=0.5, label=f"True {X_label} Samples", color="#7D8BE0")
    plt.hist(logK_Y, bins=bins, alpha=0.5, label=f"True {Y_label} Samples", color="#EA7D70")
    plt.axvline(0, color="k", linestyle="--", alpha=0.4)
    plt.xlabel(f"log K ( {Y_label} / {X_label}) ")
    plt.ylabel("Number of samples")

    # add shaded regions with text (white text on dark background)
    for pos, ((x0, x1), label) in enumerate(jeffries_logK.items()):
        plt.axvspan(x0, x1, color="gray", alpha=0.2 if pos % 2 == 0 else 0.1)
        plt.text(
            (x0 + x1) / 2,
            plt.ylim()[1] * 0.9,
            label,
            color="black" if pos % 2 == 0 else "dimgray",
            ha="center",
            va="top",
            fontsize=8,
            rotation=90,
        )

    # Add a symlog scale on the x-axis
    plt.xscale("symlog", linthresh=3)

    # Label either end of the x-axis with the corresponding model
    plt.text(
        -90,
        plt.ylim()[1] * 0.94,
        f"Evidence for {X_label}",
        color="#7D8BE0",
        fontsize=12,
        ha="left",
    )
    plt.text(
        90,
        plt.ylim()[1] * 0.94,
        f"Evidence for {Y_label}",
        color="#EA7D70",
        fontsize=12,
        ha="right",
    )

    plt.legend()

In [ ]:
%load_ext autoreload
%autoreload 2

import numpy as np

from synference import (
    SBI_Fitter,
    create_uncertainty_models_from_EPOCHS_cat,
    load_unc_model_from_hdf5,
)

In [ ]:
bpass_path = "/home/tharvey/work/synference/libraries/grid_BPASS_Chab_DenseBasis_SFH_0.01_z_14_logN_5.0_Calzetti_v3_multinode.hdf5"  # noqa: E501

fitter_bpass = SBI_Fitter.init_from_hdf5(
    "BPASS_Chab_DenseBasis_SFH_0.01_z_14_logN_5.0_Calzetti_v3_multinode", bpass_path
)

grid_pop3_path = (
    "/home/tharvey/work/synference/libraries/grid_Yggdrasil_Chab_Burst_SFH_0.001_z_14_logN_4.4_v1.hdf5"  # noqa: E501
)

fitter_pop3 = SBI_Fitter.init_from_hdf5(
    "Yggdrasil_Chab_Burst_SFH_0.001_z_14_logN_4.4_v1", grid_pop3_path
)


noise_model_path = "/home/tharvey/work/synference/priv/data/JOF_psfmatched_asinh_noise_model.h5"
noise_models = load_unc_model_from_hdf5(noise_model_path)

hst_filters = ["F435W", "F606W", "F775W", "F814W", "F850LP"]

nm = {f"JWST/NIRCam.{i}": j for i, j in noise_models.items() if i not in hst_filters}

softening_parameter = (35 * u.ABmag).to_value("Jy") * Jy


depths = {  # per pixel at 5-sigma
    "HST/ACS_WFC.F435W": 0.394 * nJy,
    "HST/ACS_WFC.F606W": 0.611 * nJy,
    "HST/ACS_WFC.F775W": 0.376 * nJy,
    "HST/ACS_WFC.F814W": 1.388 * nJy,
    "HST/ACS_WFC.F850LP": 0.724 * nJy,
    "JWST/NIRCam.F090W": 0.601 * nJy,
    "JWST/NIRCam.F115W": 0.496 * nJy,
    "JWST/NIRCam.F150W": 0.489 * nJy,
    "JWST/NIRCam.F182M": 1.361 * nJy,
    "JWST/NIRCam.F200W": 0.509 * nJy,
    "JWST/NIRCam.F210M": 0.987 * nJy,
    "JWST/NIRCam.F277W": 0.367 * nJy,
    "JWST/NIRCam.F335M": 0.616 * nJy,
    "JWST/NIRCam.F356W": 0.416 * nJy,
    "JWST/NIRCam.F410M": 0.547 * nJy,
    "JWST/NIRCam.F430M": 1.327 * nJy,
    "JWST/NIRCam.F444W": 0.472 * nJy,
    "JWST/NIRCam.F460M": 1.813 * nJy,
    "JWST/NIRCam.F480M": 1.351 * nJy,
}

fa_input = dict(
    extra_features=["redshift"],
    # empirical_noise_models=nm,
    include_errors_in_feature_array=True,
    scatter_fluxes=True,
    normed_flux_units="asinh",
    depths=depths,
    photometry_to_remove=[
        "CTIO/DECam.u",
        "CTIO/DECam.g",
        "CTIO/DECam.r",
        "CTIO/DECam.i",
        "CTIO/DECam.z",
        "CTIO/DECam.Y",
        "LSST/LSST.u",
        "LSST/LSST.g",
        "LSST/LSST.r",
        "LSST/LSST.i",
        "LSST/LSST.z",
        "LSST/LSST.Y",
        "PAN-STARRS/PS1.g",
        "PAN-STARRS/PS1.r",
        "PAN-STARRS/PS1.i",
        "PAN-STARRS/PS1.w",
        "PAN-STARRS/PS1.z",
        "PAN-STARRS/PS1.y",
        "Paranal/VISTA.Z",
        "Paranal/VISTA.Y",
        "Paranal/VISTA.J",
        "Paranal/VISTA.H",
        "Paranal/VISTA.Ks",
        "Subaru/HSC.g",
        "Subaru/HSC.r",
        "Subaru/HSC.i",
        "Subaru/HSC.z",
        "Subaru/HSC.Y",
        "CFHT/MegaCam.u",
        "CFHT/MegaCam.g",
        "CFHT/MegaCam.r",
        "CFHT/MegaCam.i",
        "CFHT/MegaCam.z",
        "Euclid/VIS.vis",
        "Euclid/NISP.Y",
        "Euclid/NISP.J",
        "Euclid/NISP.H",
        "HST/ACS_WFC.F475W",
        "HST/WFC3_IR.F105W",
        "JWST/NIRCam.F070W",
        "HST/WFC3_IR.F110W",
        "HST/WFC3_IR.F125W",
        "JWST/NIRCam.F140M",
        "HST/WFC3_IR.F140W",
        "HST/WFC3_IR.F160W",
        "JWST/NIRCam.F162M",
        "JWST/NIRCam.F250M",
        "JWST/NIRCam.F300M",
        "JWST/NIRCam.F360M",
        "JWST/NIRCam.F430M",
        "JWST/NIRCam.F460M",
        "JWST/NIRCam.F480M",
        "JWST/MIRI.F560W",
        "JWST/MIRI.F770W",
        "JWST/MIRI.F1000W",
        "JWST/MIRI.F1130W",
        "JWST/MIRI.F1280W",
        "JWST/MIRI.F1500W",
        "JWST/MIRI.F1800W",
        "JWST/MIRI.F2100W",
        "JWST/MIRI.F2550W",
        "Spitzer/IRAC.I1",
        "Spitzer/IRAC.I2",
        "Spitzer/IRAC.I3",
        "Spitzer/IRAC.I4",
        "HST/ACS_WFC.F435W",
        "HST/ACS_WFC.F606W",
        "HST/ACS_WFC.F775W",
        "HST/ACS_WFC.F814W",
        "HST/ACS_WFC.F850LP",
    ],
    asinh_softening_parameters=softening_parameter,
)

fitter_bpass.create_feature_array_from_raw_photometry(
    **fa_input, parameter_transformations={"Av": np.log10}
)

fitter_pop3.create_feature_array_from_raw_photometry(
    **fa_input,
);

In [ ]:
train_idx, test_idx = fitter_bpass.split_dataset(random_seed=42)

# Make loaders
from ili.dataloaders import NumpyLoader

train_loader_bpass = NumpyLoader(
    fitter_bpass.feature_array[train_idx].astype(np.float32),
    fitter_bpass.fitted_parameter_array[train_idx].astype(np.float32),
)
test_loader_bpass = NumpyLoader(
    fitter_bpass.feature_array[test_idx].astype(np.float32),
    fitter_bpass.fitted_parameter_array[test_idx].astype(np.float32),
)


train_idx, test_idx = fitter_pop3.split_dataset(random_seed=42)

# Make loaders

train_loader_pop3 = NumpyLoader(
    fitter_pop3.feature_array[train_idx].astype(np.float32),
    fitter_pop3.fitted_parameter_array[train_idx].astype(np.float32),
)
test_loader_pop3 = NumpyLoader(
    fitter_pop3.feature_array[test_idx].astype(np.float32),
    fitter_pop3.fitted_parameter_array[test_idx].astype(np.float32),
)

In [ ]:
network = K_EvidenceNetwork()
summary = network.train(train_loader_bpass, train_loader_pop3)

In [ ]:
# Pick 10 random samples from each test set
np.random.seed(42)

# Do all of both test sets

x_bpass = test_loader_bpass.get_all_data()
x_pop3 = test_loader_pop3.get_all_data()

logK_bpass = network.predict(x_bpass)
logK_pop3 = network.predict(x_pop3)

import matplotlib.pyplot as plt

# Clip to 100, -100
logK_bpass = np.clip(logK_bpass, -100, 100)
logK_pop3 = np.clip(logK_pop3, -100, 100)


plot_histogram(logK_bpass, logK_pop3)

# Train and test a model comparison for all Pop III models.

In [ ]:
# Try all Yggdrasil models
grids = [
    "grid_yggdrasil-1.3.3-PopIII_kroupa-0.1,100_Burst_SFH_0.001_z_14_logN_4.4_v1",
    "grid_yggdrasil-1.3.3-PopIII_salpeter-10,1,500_Burst_SFH_0.001_z_14_logN_4.4_v1",
    "grid_yggdrasil-1.3.3-PopIII_salpeter-50,500_Burst_SFH_0.001_z_14_logN_4.4_v1",
    "grid_yggdrasil-1.3.3-POPIII-fcov_0.5_kroupa-0.1,100_Burst_SFH_0.001_z_14_logN_4.4_v1",
    "grid_yggdrasil-1.3.3-POPIII-fcov_0.5_salpeter-10,1,500_Burst_SFH_0.001_z_14_logN_4.4_v1",
    "grid_yggdrasil-1.3.3-POPIII-fcov_0.5_salpeter-50,500_Burst_SFH_0.001_z_14_logN_4.4_v1",
    "grid_yggdrasil-1.3.3-POPIII-fcov_1_kroupa-0.1,100_Burst_SFH_0.001_z_14_logN_4.4_v1",
    "grid_yggdrasil-1.3.3-POPIII-fcov_1_salpeter-10,1,500_Burst_SFH_0.001_z_14_logN_4.4_v1",
    "grid_yggdrasil-1.3.3-POPIII-fcov_1_salpeter-50,500_Burst_SFH_0.001_z_14_logN_4.4_v1",
]

for grid in grids:
    print(f"Processing grid: {grid}")
    fitter_pop3 = SBI_Fitter.init_from_hdf5(
        grid,
        f"{grid}.hdf5",
    )
    fitter_pop3.create_feature_array_from_raw_photometry(
        **fa_input, parameter_transformations={"sfh": np.log10}
    )

    train_idx, test_idx = fitter_pop3.split_dataset(random_seed=42)
    train_loader_pop3 = NumpyLoader(
        fitter_pop3.feature_array[train_idx].astype(np.float32),
        fitter_pop3.fitted_parameter_array[train_idx].astype(np.float32),
    )

    test_loader_pop3 = NumpyLoader(
        fitter_pop3.feature_array[test_idx].astype(np.float32),
        fitter_pop3.fitted_parameter_array[test_idx].astype(np.float32),
    )

    network = K_EvidenceNetwork()
    summary = network.train(train_loader_bpass, train_loader_pop3)
    x_pop3 = test_loader_pop3.get_all_data()
    x_bpass = test_loader_bpass.get_all_data()
    logK_pop3 = network.predict(x_pop3)
    logK_bpass = network.predict(x_bpass)

    logK_pop3 = np.clip(logK_pop3, -100, 100)
    logK_bpass = np.clip(logK_bpass, -100, 100)
    plot_histogram(logK_bpass, logK_pop3)
    plt.title(grid)
    plt.savefig(f"/home/tharvey/work/synference/priv/pop3_bpass_logK_{grid}.png")

# FSPS vs BPASS

In [ ]:
bpass_path = "/home/tharvey/work/synference/libraries/grid_BPASS_Chab_DenseBasis_SFH_0.01_z_14_logN_5.0_Calzetti_v3_multinode.hdf5"  # noqa: E501

fitter_bpass = SBI_Fitter.init_from_hdf5(
    "BPASS_Chab_DenseBasis_SFH_0.01_z_14_logN_5.0_Calzetti_v3_multinode", bpass_path
)

grid_fsps = "/home/tharvey/work/synference/libraries/grid_FSPS_Chab_DenseBasis_SFH_0.01_z_14_logN_6.0_Calzetti_v4_multinode.hdf5"  # noqa: E501

fitter_fsps = SBI_Fitter.init_from_hdf5(
    "FSPS_Chab_DenseBasis_SFH_0.01_z_14_logN_6.0_Calzetti_v4_multinode", grid_fsps
)

bands = [
    "F435W",
    "F606W",
    "F775W",
    "F814W",
    "F850LP",
    "F090W",
    "F115W",
    "F150W",
    "F200W",
    "F277W",
    "F335M",
    "F356W",
    "F410M",
    "F444W",
]

hst_bands = ["F435W", "F606W", "F775W", "F814W", "F850LP"]
new_band_names = [
    (f"HST/ACS_WFC.{band.upper()}" if band in hst_bands else f"JWST/NIRCam.{band.upper()}")
    for band in bands
]
data_err_file = "/home/tharvey/work/JADES-DR3-GS_MASTER_Sel-F277W+F356W+F444W_v13.fits"


empirical_noise_models = create_uncertainty_models_from_EPOCHS_cat(
    data_err_file,
    bands,
    new_band_names,
    plot=False,
    hdu="OBJECTS",
    save=False,
    min_flux_error=0,
    model_class="asinh",
)


fa_input = dict(
    extra_features=["redshift"],
    empirical_noise_models=empirical_noise_models,
    include_errors_in_feature_array=True,
    scatter_fluxes=True,
    normed_flux_units="asinh",
    photometry_to_remove=[
        "CTIO/DECam.u",
        "CTIO/DECam.g",
        "CTIO/DECam.r",
        "CTIO/DECam.i",
        "CTIO/DECam.z",
        "CTIO/DECam.Y",
        "LSST/LSST.u",
        "LSST/LSST.g",
        "LSST/LSST.r",
        "LSST/LSST.i",
        "LSST/LSST.z",
        "LSST/LSST.Y",
        "PAN-STARRS/PS1.g",
        "PAN-STARRS/PS1.r",
        "PAN-STARRS/PS1.i",
        "PAN-STARRS/PS1.w",
        "PAN-STARRS/PS1.z",
        "PAN-STARRS/PS1.y",
        "Paranal/VISTA.Z",
        "Paranal/VISTA.Y",
        "Paranal/VISTA.J",
        "Paranal/VISTA.H",
        "Paranal/VISTA.Ks",
        "Subaru/HSC.g",
        "Subaru/HSC.r",
        "Subaru/HSC.i",
        "Subaru/HSC.z",
        "Subaru/HSC.Y",
        "CFHT/MegaCam.u",
        "CFHT/MegaCam.g",
        "CFHT/MegaCam.r",
        "CFHT/MegaCam.i",
        "CFHT/MegaCam.z",
        "Euclid/VIS.vis",
        "Euclid/NISP.Y",
        "Euclid/NISP.J",
        "Euclid/NISP.H",
        "HST/ACS_WFC.F475W",
        "HST/WFC3_IR.F105W",
        "JWST/NIRCam.F070W",
        "HST/WFC3_IR.F110W",
        "HST/WFC3_IR.F125W",
        "JWST/NIRCam.F140M",
        "HST/WFC3_IR.F140W",
        "HST/WFC3_IR.F160W",
        "JWST/NIRCam.F162M",
        "JWST/NIRCam.F182M",
        "JWST/NIRCam.F210M",
        "JWST/NIRCam.F250M",
        "JWST/NIRCam.F300M",
        "JWST/NIRCam.F360M",
        "JWST/NIRCam.F430M",
        "JWST/NIRCam.F460M",
        "JWST/NIRCam.F480M",
        "JWST/MIRI.F560W",
        "JWST/MIRI.F770W",
        "JWST/MIRI.F1000W",
        "JWST/MIRI.F1130W",
        "JWST/MIRI.F1280W",
        "JWST/MIRI.F1500W",
        "JWST/MIRI.F1800W",
        "JWST/MIRI.F2100W",
        "JWST/MIRI.F2550W",
        "Spitzer/IRAC.I1",
        "Spitzer/IRAC.I2",
        "Spitzer/IRAC.I3",
        "Spitzer/IRAC.I4",
    ],
)

fitter_bpass.create_feature_array_from_raw_photometry(
    **fa_input,
)

fitter_fsps.create_feature_array_from_raw_photometry(
    **fa_input,
    max_rows=100_000,
);

In [ ]:
train_idx, test_idx = fitter_bpass.split_dataset(random_seed=42)

# Make loaders
from ili.dataloaders import NumpyLoader

train_loader_bpass = NumpyLoader(
    fitter_bpass.feature_array[train_idx].astype(np.float32),
    fitter_bpass.fitted_parameter_array[train_idx].astype(np.float32),
)
test_loader_bpass = NumpyLoader(
    fitter_bpass.feature_array[test_idx].astype(np.float32),
    fitter_bpass.fitted_parameter_array[test_idx].astype(np.float32),
)


train_idx, test_idx = fitter_fsps.split_dataset(random_seed=42)

# Make loaders

train_loader_fsps = NumpyLoader(
    fitter_fsps.feature_array[train_idx].astype(np.float32),
    fitter_fsps.fitted_parameter_array[train_idx].astype(np.float32),
)
test_loader_fsps = NumpyLoader(
    fitter_fsps.feature_array[test_idx].astype(np.float32),
    fitter_fsps.fitted_parameter_array[test_idx].astype(np.float32),
)

In [ ]:
network = K_EvidenceNetwork(device="cuda", layer_width=32, added_layers=5)
summary = network.train(train_loader_bpass, train_loader_fsps)

In [ ]:
# Pick 10 random samples from each test set
np.random.seed(42)

# Do all of both test sets

x_bpass = test_loader_bpass.get_all_data()
x_fsps = test_loader_fsps.get_all_data()

logK_bpass = network.predict(x_bpass)
logK_fsps = network.predict(x_fsps)

import matplotlib.pyplot as plt

# Clip to 100, -100
logK_bpass = np.clip(logK_bpass, -100, 100)
logK_fsps = np.clip(logK_fsps, -100, 100)


plot_histogram(logK_bpass, logK_fsps, X_label="BPASS", Y_label="FSPS")
plt.yscale("log")